In [ ]:
# default_exp model.metrics

# Metrics

> A collection of Metrics used in the segmentation models

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export
from steel_segmentation.core import *
from steel_segmentation.data import *
from steel_segmentation.preprocessing import *

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    from fastai.vision.all import *
    import fastai
from fastcore.foundation import *

## Fastai metrics

In [ ]:
# export
class DiceMulti(Metric):
    "Averaged Dice metric (Macro F1) for multiclass target in segmentation"

    def __init__(self, axis=1): self.axis = axis
    def reset(self): self.inter, self.union = {}, {}

    def accumulate(self, learn):
        pred, targ = flatten_check(learn.pred.argmax(dim=self.axis), learn.y)
        for c in range(learn.pred.shape[self.axis]):
            p = torch.where(pred == c, 1, 0)
            t = torch.where(targ == c, 1, 0)
            p, t = TensorBase(p), TensorBase(t)
            c_inter = (p*t).float().sum().item()
            c_union = (p+t).float().sum().item()
            if c in self.inter:
                self.inter[c] += c_inter
                self.union[c] += c_union
            else:
                self.inter[c] = c_inter
                self.union[c] = c_union

    @property
    def value(self):
        binary_dice_scores = np.array([])
        for c in self.inter:
            binary_dice_scores = np.append(
                binary_dice_scores, 2.*self.inter[c]/self.union[c] if self.union[c] > 0 else np.nan)
        return np.nanmean(binary_dice_scores)

In [ ]:
# export
def dice_kaggle(input: Tensor, targs: Tensor, iou: bool = False, eps: float = 1e-8):
    """From https://www.kaggle.com/iafoss/severstal-fast-ai-256x256-crops"""
    n, c = targs.shape[0], input.shape[1]
    input = input.argmax(dim=1).view(n, -1)
    targs = targs.view(n, -1)

    intersect, union = [], []
    for i in range(1, c):
        inp, trgs = TensorBase(input), TensorBase(targs)
        intersect.append(((inp == i) & (trgs == i)).sum(-1).float())
        union.append(((inp == i).sum(-1) + (trgs == i).sum(-1)).float())

    intersect = torch.stack(intersect)
    union = torch.stack(union)

    if not iou:
        return ((2.0*intersect + eps) / (union+eps)).mean()
    else:
        return ((intersect + eps) / (union - intersect + eps)).mean()

In [ ]:
# hide
def dice_forum(input: Tensor, targs: Tensor, eps: float = 1e-8):
    """https://forums.fast.ai/t/training-unet-for-segmentation-negative-dice-score/45137/9"""
    n = targs.shape[0]  # channels
    targs = targs.squeeze(1)  # add batch dimension

    input = input.argmax(dim=1).view(n, -1)

    targs = targs.view(n, -1)
    targs1 = (targs > 0).float()
    input1 = (input > 0).float()

    ss = (input == targs).float()
    intersect = (ss * targs1).sum(dim=1).float()
    union = (input1+targs1).sum(dim=1).float()

    l = 2. * intersect / union
    l[union == 0.] = 1.

    return l.mean()

## Kaggle metrics

In [ ]:
# export
def metric(probability, truth, threshold=0.5, reduction='none'):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = (truth > 0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])

#         dice_neg = np.nan_to_num(dice_neg.mean().item(), 0)
#         dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)
#         dice = dice.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()